In [1]:
import os
os.getcwd()
%cd MuseTalk/

/home/ldap-users-2/lathifgalih-k/clone-video/muse-exp/MuseTalk


In [2]:
import sys
print(sys.executable)

/home/is/lathifgalih-k/miniconda3/envs/muse-talk-bu/bin/python


In [3]:
## Preprocessing video for MuseTalk
import cv2
import os
import subprocess
import soundfile as sf
import numpy as np
from moviepy.editor import VideoFileClip

### Sample Audio

In [ ]:
# Set your input video path and output directory
input_video_path = "/home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video/muse-exp/MuseTalk/data/video/A002_02_BBP_NORMAL_color.avi"
output_dir = "/home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video/muse-exp/MuseTalk/data/video/"
os.makedirs(output_dir, exist_ok=True)

video_name = os.path.splitext(os.path.basename(input_video_path))[0]
output_path = os.path.join(output_dir, f"{video_name}_25fps_6sec.mp4")

In [5]:
# get video properties
cap = cv2.VideoCapture(input_video_path)
original_fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
duration = frame_count / original_fps
cap.release()

print(f"Video asli:")
print(f"FPS: {original_fps}")
print(f"Durasi: {duration:.2f} detik")
print(f"Total frame: {frame_count}")

Video asli:
FPS: 50.18165676442798
Durasi: 24.41 detik
Total frame: 1225


In [8]:
# Cell untuk fix ffmpeg path
import os
import subprocess

# Set ffmpeg path yang benar
ffmpeg_dir = "/home/is/lathifgalih-k/musetalk/ffmpeg-4.4-amd64-static"
ffmpeg_executable = os.path.join(ffmpeg_dir, "ffmpeg")

# Cek apakah ffmpeg executable ada
if os.path.exists(ffmpeg_executable):
    print(f"✓ FFmpeg executable ditemukan: {ffmpeg_executable}")
    
    # Test ffmpeg
    try:
        result = subprocess.run([ffmpeg_executable, '-version'], capture_output=True, text=True)
        if result.returncode == 0:
            print("✓ FFmpeg berfungsi!")
            print(result.stdout.split('\n')[0])
        else:
            print("✗ FFmpeg error")
    except Exception as e:
        print(f"✗ Error: {e}")
else:
    print(f"✗ FFmpeg executable tidak ditemukan di: {ffmpeg_executable}")
    # List isi direktori untuk cek
    if os.path.exists(ffmpeg_dir):
        print(f"Isi direktori {ffmpeg_dir}:")
        for item in os.listdir(ffmpeg_dir):
            print(f"  {item}")

✓ FFmpeg executable ditemukan: /home/is/lathifgalih-k/musetalk/ffmpeg-4.4-amd64-static/ffmpeg
✓ FFmpeg berfungsi!
ffmpeg version 7.0.2-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2024 the FFmpeg developers


In [10]:
# Process video dengan ffmpeg path yang benar
start_time = 0  # Mulai dari detik ke-0, bisa diganti
target_duration = 6  # 6 detik
target_fps = 25

# Set ffmpeg executable path yang benar
ffmpeg_executable = "/home/is/lathifgalih-k/musetalk/ffmpeg-4.4-amd64-static/ffmpeg"

# Command ffmpeg untuk process video
cmd = [
    ffmpeg_executable,  # Gunakan path lengkap ke executable
    '-i', input_video_path,
    '-ss', str(start_time),          # Start time
    '-t', str(target_duration),      # Duration 6 detik  
    '-r', str(target_fps),           # Output FPS 25
    '-c:v', 'libx264',               # Video codec
    '-c:a', 'aac',                   # Audio codec
    '-y',                            # Overwrite jika file sudah ada
    output_path
]

print("Processing video...")
print(" ".join(cmd))
result = subprocess.run(cmd, capture_output=True, text=True)

if result.returncode == 0:
    print(f"✓ Berhasil! Video disimpan di: {output_path}")
else:
    print(f"✗ Error: {result.stderr}")
    print(f"✗ Output: {result.stdout}")

Processing video...
/home/is/lathifgalih-k/musetalk/ffmpeg-4.4-amd64-static/ffmpeg -i /home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video/muse-exp/MuseTalk/data/video/A002_02_BBP_NORMAL_color.avi -ss 0 -t 6 -r 25 -c:v libx264 -c:a aac -y /home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video/muse-exp/MuseTalk/data/process_videos/A002_02_BBP_NORMAL_color_25fps_6sec.mp4
✓ Berhasil! Video disimpan di: /home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video/muse-exp/MuseTalk/data/process_videos/A002_02_BBP_NORMAL_color_25fps_6sec.mp4


In [11]:
# Cek hasil video yang sudah diproses
if os.path.exists(output_path):
    cap_final = cv2.VideoCapture(output_path)
    final_fps = cap_final.get(cv2.CAP_PROP_FPS)
    final_frame_count = int(cap_final.get(cv2.CAP_PROP_FRAME_COUNT))
    final_duration = final_frame_count / final_fps if final_fps > 0 else 0
    cap_final.release()

    print(f"Video hasil processing:")
    print(f"Path: {output_path}")
    print(f"FPS: {final_fps}")
    print(f"Durasi: {final_duration:.2f} detik")
    print(f"Total frame: {final_frame_count}")
    print(f"✓ File siap untuk MuseTalk!")
else:
    print("File output tidak ditemukan!")

Video hasil processing:
Path: /home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video/muse-exp/MuseTalk/data/process_videos/A002_02_BBP_NORMAL_color_25fps_6sec.mp4
FPS: 25.0
Durasi: 6.00 detik
Total frame: 150
✓ File siap untuk MuseTalk!


### Preprocessing for all ALS video

In [4]:
input_audio = "/home/is/lathifgalih-k/research_naist/multimodal-clone/clone-voice-coqui/output_audio/RAW/ALS/XTTSv2"
speaker_id = []
for file in os.listdir(input_audio):
    if file.endswith("wav"):
        speaker_id.append(file.split("_")[0])

print(f"Speaker IDs: {speaker_id}")

Speaker IDs: ['A017', 'A006', 'A010', 'A008', 'A015', 'A002', 'A014', 'A011', 'A016']


In [5]:
input_video = "/home/is/lathifgalih-k/research_naist/multimodal-clone/dataset-toronto/ALS/Videos"
output_dir = "/home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video/muse-exp/MuseTalk/data/video/als-input"

# List all video files in the input directory
video_files = []
for file in os.listdir(input_video):
    sid = file.split("_")[0]
    if sid in speaker_id and file.endswith(".mp4"):
        if sid == "A015" and "PATAKA_" in file.upper():
            for i in video_files:
                if sid in i:
                    print(f"File {file} sudah ada di daftar video.")
                    break
            else:
                print(f"Menambahkan file {file} ke daftar video.")
                video_files.append(file)
        elif sid != "A015" and "BBP_NORMAL" in file.upper():
            for i in video_files:
                if sid in i:
                    print(f"File {file} sudah ada di daftar video.")
                    break
            else:
                print(f"Menambahkan file {file} ke daftar video.")
                video_files.append(file)
print(f"Video files to process: {video_files}")
print(f"Total video files found: {len(video_files)}")

Menambahkan file A017_02_BBP_NORMAL_color.mp4 ke daftar video.
Menambahkan file A010_02_BBP_NORMAL_color.mp4 ke daftar video.
Menambahkan file A014_02_BBP_NORMAL_color.mp4 ke daftar video.
Menambahkan file A008_02_BBP_NORMAL_color.mp4 ke daftar video.
Menambahkan file A002_02_BBP_NORMAL_color.mp4 ke daftar video.
Menambahkan file A006_02_BBP_NORMAL_color.mp4 ke daftar video.
Menambahkan file A016_02_BBP_NORMAL_color.mp4 ke daftar video.
Menambahkan file A015_02_DDK_PATAKA_color.mp4 ke daftar video.
Menambahkan file A011_02_BBP_NORMAL_color.mp4 ke daftar video.
Video files to process: ['A017_02_BBP_NORMAL_color.mp4', 'A010_02_BBP_NORMAL_color.mp4', 'A014_02_BBP_NORMAL_color.mp4', 'A008_02_BBP_NORMAL_color.mp4', 'A002_02_BBP_NORMAL_color.mp4', 'A006_02_BBP_NORMAL_color.mp4', 'A016_02_BBP_NORMAL_color.mp4', 'A015_02_DDK_PATAKA_color.mp4', 'A011_02_BBP_NORMAL_color.mp4']
Total video files found: 9


In [6]:
# get video properties
for file in video_files:
    input_video_path = os.path.join(input_video, file)
    cap = cv2.VideoCapture(input_video_path)
    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count / original_fps
    cap.release()
    print(f"\nProcessing video: {file}")
    print(f"Video asli:")
    print(f"FPS: {original_fps}")
    print(f"Durasi: {duration:.2f} detik")
    print(f"Total frame: {frame_count}")


Processing video: A017_02_BBP_NORMAL_color.mp4
Video asli:
FPS: 25.0
Durasi: 32.12 detik
Total frame: 803

Processing video: A010_02_BBP_NORMAL_color.mp4
Video asli:
FPS: 25.0
Durasi: 49.60 detik
Total frame: 1240

Processing video: A014_02_BBP_NORMAL_color.mp4
Video asli:
FPS: 25.0
Durasi: 34.60 detik
Total frame: 865

Processing video: A008_02_BBP_NORMAL_color.mp4
Video asli:
FPS: 25.0
Durasi: 48.08 detik
Total frame: 1202

Processing video: A002_02_BBP_NORMAL_color.mp4
Video asli:
FPS: 25.0
Durasi: 24.48 detik
Total frame: 612

Processing video: A006_02_BBP_NORMAL_color.mp4
Video asli:
FPS: 25.0
Durasi: 32.04 detik
Total frame: 801

Processing video: A016_02_BBP_NORMAL_color.mp4
Video asli:
FPS: 25.0
Durasi: 31.88 detik
Total frame: 797

Processing video: A015_02_DDK_PATAKA_color.mp4
Video asli:
FPS: 25.0
Durasi: 22.80 detik
Total frame: 570

Processing video: A011_02_BBP_NORMAL_color.mp4
Video asli:
FPS: 25.0
Durasi: 30.36 detik
Total frame: 759


In [7]:
## processing video files
start_time = 9
target_duration = 6
target_fps = 25

# Set ffmpeg executable path yang benar
ffmpeg_executable = "/home/is/lathifgalih-k/musetalk/ffmpeg-4.4-amd64-static/ffmpeg"

# Command ffmpeg untuk process video
for file in video_files:
    if "A008" in file:
        start_time = 12
    input_video_path = os.path.join(input_video, file)
    output_path = os.path.join(output_dir, f"{os.path.splitext(file)[0]}_25fps_6sec.mp4")
    cmd = [
        ffmpeg_executable,  # Gunakan path lengkap ke executable
        '-i', input_video_path,
        '-ss', str(start_time),          # Start time
        '-t', str(target_duration),      # Duration 6 detik  
        '-r', str(target_fps),           # Output FPS 25
        '-c:v', 'libx264',               # Video codec
        '-c:a', 'aac',                   # Audio codec
        '-y',                            # Overwrite jika file sudah ada
        output_path
    ]

    print(f"\nProcessing video...")
    print(" ".join(cmd))
    result = subprocess.run(cmd, capture_output=True, text=True)

    if result.returncode == 0:
        print(f"✓ Berhasil! Video disimpan di: {output_path}")
    else:
        print(f"✗ Error: {result.stderr}")
        print(f"✗ Output: {result.stdout}")


Processing video...
/home/is/lathifgalih-k/musetalk/ffmpeg-4.4-amd64-static/ffmpeg -i /home/is/lathifgalih-k/research_naist/multimodal-clone/dataset-toronto/ALS/Videos/A017_02_BBP_NORMAL_color.mp4 -ss 9 -t 6 -r 25 -c:v libx264 -c:a aac -y /home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video/muse-exp/MuseTalk/data/video/als-input/A017_02_BBP_NORMAL_color_25fps_6sec.mp4
✓ Berhasil! Video disimpan di: /home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video/muse-exp/MuseTalk/data/video/als-input/A017_02_BBP_NORMAL_color_25fps_6sec.mp4

Processing video...
/home/is/lathifgalih-k/musetalk/ffmpeg-4.4-amd64-static/ffmpeg -i /home/is/lathifgalih-k/research_naist/multimodal-clone/dataset-toronto/ALS/Videos/A010_02_BBP_NORMAL_color.mp4 -ss 9 -t 6 -r 25 -c:v libx264 -c:a aac -y /home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video/muse-exp/MuseTalk/data/video/als-input/A010_02_BBP_NORMAL_color_25fps_6sec.mp4
✓ Berhasil! Video disimpan di: /home/is/lathifgalih

### Preprocesing for all Stroke video

In [10]:
input_audio = "/home/is/lathifgalih-k/research_naist/multimodal-clone/clone-voice-coqui/output_audio/RAW/Stroke/XTTSv2"
speaker_id = []
for file in os.listdir(input_audio):
    if file.endswith("wav"):
        speaker_id.append(file.split("_")[0])

print(f"Speaker IDs: {speaker_id}")

Speaker IDs: ['S011', 'S007', 'S005', 'S013', 'S002', 'S009', 'S003', 'S008', 'S012', 'S001', 'S006']


In [18]:
input_video = "/home/is/lathifgalih-k/research_naist/multimodal-clone/dataset-toronto/Stroke/Videos"
output_dir = "/home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video/muse-exp/MuseTalk/data/video/stroke-input"

# List all video files in the input directory
video_files = []
for file in os.listdir(input_video):
    # print(file)
    sid = file.split("_")[0]
    if sid in speaker_id:
        if (sid == "S001" or sid == "S002") and "PA_" in file.upper():
            for i in video_files:
                if sid in i:
                    print(f"File {file} sudah ada di daftar video.")
                    break
            else:
                print(f"Menambahkan file {file} ke daftar video.")
                video_files.append(file)
        elif sid.startswith("S") and "BBP_NORMAL" in file.upper():
            if sid != "S001" and sid != "S002":
                for i in video_files:
                    if sid in i:
                        print(f"File {file} sudah ada di daftar video.")
                        break
                else:
                    print(f"Menambahkan file {file} ke daftar video.")
                    video_files.append(file)
print(f"Video files to process: {video_files}")
print(f"Total video files found: {len(video_files)}")

Menambahkan file S006_02_BBP_NORMAL_color.avi ke daftar video.
Menambahkan file S012_02_BBP_NORMAL_color.avi ke daftar video.
Menambahkan file S009_02_BBP_NORMAL_color.avi ke daftar video.
Menambahkan file S005_02_BBP_NORMAL_color.avi ke daftar video.
Menambahkan file S011_02_BBP_NORMAL_color.avi ke daftar video.
Menambahkan file S001_02_DDK_PA_color.avi ke daftar video.
Menambahkan file S007_02_BBP_NORMAL_color.avi ke daftar video.
Menambahkan file S003_02_BBP_NORMAL_color.avi ke daftar video.
Menambahkan file S013_02_BBP_NORMAL_color.avi ke daftar video.
Menambahkan file S002_02_DDK_PA_color.avi ke daftar video.
Menambahkan file S008_02_BBP_NORMAL_color.avi ke daftar video.
Video files to process: ['S006_02_BBP_NORMAL_color.avi', 'S012_02_BBP_NORMAL_color.avi', 'S009_02_BBP_NORMAL_color.avi', 'S005_02_BBP_NORMAL_color.avi', 'S011_02_BBP_NORMAL_color.avi', 'S001_02_DDK_PA_color.avi', 'S007_02_BBP_NORMAL_color.avi', 'S003_02_BBP_NORMAL_color.avi', 'S013_02_BBP_NORMAL_color.avi', 'S002_

In [19]:
# get video properties
for file in video_files:
    input_video_path = os.path.join(input_video, file)
    cap = cv2.VideoCapture(input_video_path)
    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count / original_fps
    cap.release()
    print(f"\nProcessing video: {file}")
    print(f"Video asli:")
    print(f"FPS: {original_fps}")
    print(f"Durasi: {duration:.2f} detik")
    print(f"Total frame: {frame_count}")


Processing video: S006_02_BBP_NORMAL_color.avi
Video asli:
FPS: 48.47320093250734
Durasi: 37.55 detik
Total frame: 1820

Processing video: S012_02_BBP_NORMAL_color.avi
Video asli:
FPS: 48.48119803540723
Durasi: 17.02 detik
Total frame: 825

Processing video: S009_02_BBP_NORMAL_color.avi
Video asli:
FPS: 29.928887084423785
Durasi: 21.35 detik
Total frame: 639

Processing video: S005_02_BBP_NORMAL_color.avi
Video asli:
FPS: 48.48686049125478
Durasi: 34.42 detik
Total frame: 1669

Processing video: S011_02_BBP_NORMAL_color.avi
Video asli:
FPS: 48.47530659200577
Durasi: 21.80 detik
Total frame: 1057

Processing video: S001_02_DDK_PA_color.avi
Video asli:
FPS: 50.25916173584837
Durasi: 12.16 detik
Total frame: 611

Processing video: S007_02_BBP_NORMAL_color.avi
Video asli:
FPS: 48.47187412429728
Durasi: 23.06 detik
Total frame: 1118

Processing video: S003_02_BBP_NORMAL_color.avi
Video asli:
FPS: 48.44870229247018
Durasi: 25.64 detik
Total frame: 1242

Processing video: S013_02_BBP_NORMAL_

In [28]:
## Processing video files for Stroke dataset
start_time = 4
target_duration = 6
target_fps = 25

# Set ffmpeg executable path yang benar
ffmpeg_executable = "/home/is/lathifgalih-k/musetalk/ffmpeg-4.4-amd64-static/ffmpeg"

# Create output directory if not exists
os.makedirs(output_dir, exist_ok=True)

# Command ffmpeg untuk process video
for file in video_files:
    # Set start time yang berbeda untuk beberapa speaker
    current_start_time = start_time
    if "S013" in file: 
        current_start_time = 9
    elif "S002" in file:
        current_start_time = 4.5

    input_video_path = os.path.join(input_video, file)
    output_path = os.path.join(output_dir, f"{os.path.splitext(file)[0]}_25fps_6sec.mp4")
    
    cmd = [
        ffmpeg_executable,  # Gunakan path lengkap ke executable
        '-i', input_video_path,
        '-ss', str(current_start_time),  # Start time
        '-t', str(target_duration),      # Duration 6 detik  
        '-r', str(target_fps),           # Output FPS 25
        '-c:v', 'libx264',               # Video codec
        '-c:a', 'aac',                   # Audio codec
        '-y',                            # Overwrite jika file sudah ada
        output_path
    ]

    print(f"\nProcessing video: {file}")
    print(" ".join(cmd))
    result = subprocess.run(cmd, capture_output=True, text=True)

    if result.returncode == 0:
        print(f"✓ Berhasil! Video disimpan di: {output_path}")
        
        # Verify hasil video
        if os.path.exists(output_path):
            cap_check = cv2.VideoCapture(output_path)
            check_fps = cap_check.get(cv2.CAP_PROP_FPS)
            check_frame_count = int(cap_check.get(cv2.CAP_PROP_FRAME_COUNT))
            check_duration = check_frame_count / check_fps if check_fps > 0 else 0
            cap_check.release()
            
            print(f"  - FPS: {check_fps}")
            print(f"  - Durasi: {check_duration:.2f} detik")
            print(f"  - Total frame: {check_frame_count}")
        
    else:
        print(f"✗ Error processing {file}:")
        print(f"  stderr: {result.stderr}")
        print(f"  stdout: {result.stdout}")

print(f"\n=== Processing Stroke videos completed ===")
print(f"Output directory: {output_dir}")


Processing video: S006_02_BBP_NORMAL_color.avi
/home/is/lathifgalih-k/musetalk/ffmpeg-4.4-amd64-static/ffmpeg -i /home/is/lathifgalih-k/research_naist/multimodal-clone/dataset-toronto/Stroke/Videos/S006_02_BBP_NORMAL_color.avi -ss 4 -t 6 -r 25 -c:v libx264 -c:a aac -y /home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video/muse-exp/MuseTalk/data/video/stroke-input/S006_02_BBP_NORMAL_color_25fps_6sec.mp4
✓ Berhasil! Video disimpan di: /home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video/muse-exp/MuseTalk/data/video/stroke-input/S006_02_BBP_NORMAL_color_25fps_6sec.mp4
  - FPS: 25.0
  - Durasi: 6.00 detik
  - Total frame: 150

Processing video: S012_02_BBP_NORMAL_color.avi
/home/is/lathifgalih-k/musetalk/ffmpeg-4.4-amd64-static/ffmpeg -i /home/is/lathifgalih-k/research_naist/multimodal-clone/dataset-toronto/Stroke/Videos/S012_02_BBP_NORMAL_color.avi -ss 4 -t 6 -r 25 -c:v libx264 -c:a aac -y /home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video/muse-e

### Preprocesing Input Audio - RAW

In [9]:
import os
import shutil

# Define paths
input_folder_audio = "/home/is/lathifgalih-k/research_naist/multimodal-clone/clone-voice-coqui/output_audio/RAW"
output_base_folder = "/home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video/muse-exp/MuseTalk/data/audio"

# Define mapping for prefixes
prefix_mapping = {
    "CosyVoice2": "CV_",
    "XTTSv2": "XT_", 
    "YourTTS": "YT_"
}

# Define datasets to process
datasets = ["ALS", "Stroke"]

# Process each dataset
for dataset in datasets:
    print(f"\n=== Processing {dataset} dataset ===")
    
    # Define input and output folders
    input_dataset_folder = os.path.join(input_folder_audio, dataset)
    output_dataset_folder = os.path.join(output_base_folder, f"{dataset}-Raw")
    
    # Create output dataset folder if it doesn't exist
    os.makedirs(output_dataset_folder, exist_ok=True)
    
    # Process each voice cloning method
    for method in ["CosyVoice2", "XTTSv2", "YourTTS"]:
        print(f"\nProcessing {method}...")
        
        # Define input and output method folders
        input_method_folder = os.path.join(input_dataset_folder, method)
        output_method_folder = os.path.join(output_dataset_folder, method)
        
        # Create output method folder if it doesn't exist
        os.makedirs(output_method_folder, exist_ok=True)
        
        # Check if input method folder exists
        if not os.path.exists(input_method_folder):
            print(f"  ⚠️ Input folder tidak ditemukan: {input_method_folder}")
            continue
            
        # Process each audio file
        audio_files = [f for f in os.listdir(input_method_folder) if f.endswith('.wav')]
        
        if not audio_files:
            print(f"  ⚠️ Tidak ada file WAV ditemukan di: {input_method_folder}")
            continue
            
        for audio_file in audio_files:
            # Get the prefix for this method
            prefix = prefix_mapping[method]
            
            # Create new filename with prefix
            if dataset == "ALS":
                # For ALS: A002_cloned.wav -> CV_A002_BBP_cloned.wav (except A015, use PATAKA)
                speaker_id = audio_file.split("_")[0]
                if speaker_id == "A015":
                    new_filename = f"{prefix}{speaker_id}_PATAKA_cloned.wav"
                else:
                    new_filename = f"{prefix}{speaker_id}_BBP_cloned.wav"
            else:
                # For Stroke: S001_cloned.wav -> CV_S001_cloned.wav
                speaker_id = audio_file.split("_")[0]
                if speaker_id == "S001" or speaker_id == "S002":
                    new_filename = f"{prefix}{speaker_id}_PA_cloned.wav"
                else:
                    new_filename = f"{prefix}{speaker_id}_BBP_cloned.wav"

            # Define source and destination paths
            src_path = os.path.join(input_method_folder, audio_file)
            dst_path = os.path.join(output_method_folder, new_filename)
            
            # Copy file with new name
            try:
                shutil.copy2(src_path, dst_path)
                print(f"  ✓ {audio_file} -> {new_filename}")
            except Exception as e:
                print(f"  ✗ Error copying {audio_file}: {e}")


=== Processing ALS dataset ===

Processing CosyVoice2...
  ✓ A016_cloned.wav -> CV_A016_BBP_cloned.wav
  ✓ A011_cloned.wav -> CV_A011_BBP_cloned.wav
  ✓ A002_cloned.wav -> CV_A002_BBP_cloned.wav
  ✓ A014_cloned.wav -> CV_A014_BBP_cloned.wav
  ✓ A008_cloned.wav -> CV_A008_BBP_cloned.wav
  ✓ A015_cloned.wav -> CV_A015_PATAKA_cloned.wav
  ✓ A006_cloned.wav -> CV_A006_BBP_cloned.wav
  ✓ A010_cloned.wav -> CV_A010_BBP_cloned.wav
  ✓ A017_cloned.wav -> CV_A017_BBP_cloned.wav

Processing XTTSv2...
  ✓ A017_cloned.wav -> XT_A017_BBP_cloned.wav
  ✓ A006_cloned.wav -> XT_A006_BBP_cloned.wav
  ✓ A010_cloned.wav -> XT_A010_BBP_cloned.wav
  ✓ A008_cloned.wav -> XT_A008_BBP_cloned.wav
  ✓ A015_cloned.wav -> XT_A015_PATAKA_cloned.wav
  ✓ A002_cloned.wav -> XT_A002_BBP_cloned.wav
  ✓ A014_cloned.wav -> XT_A014_BBP_cloned.wav
  ✓ A011_cloned.wav -> XT_A011_BBP_cloned.wav
  ✓ A016_cloned.wav -> XT_A016_BBP_cloned.wav

Processing YourTTS...
  ✓ A014_cloned.wav -> YT_A014_BBP_cloned.wav
  ✓ A002_cloned.w

### Preprocessing Audio Data for VCTK

In [2]:
import os
import shutil

# Define paths
input_folder_audio = "/home/is/lathifgalih-k/research_naist/multimodal-clone/clone-voice-coqui/output_audio/VCTK"
output_base_folder = "/home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video/muse-exp/MuseTalk/data/audio"

# Define mapping for prefixes
prefix_mapping = {
    "CosyVoice2": "CV_",
    "XTTSv2": "XT_", 
    "YourTTS": "YT_"
}

# Define datasets to process
datasets = ["ALS", "Stroke"]

# Process each dataset
for dataset in datasets:
    print(f"\n=== Processing {dataset} dataset ===")
    
    # Define input and output folders
    input_dataset_folder = os.path.join(input_folder_audio, dataset)
    output_dataset_folder = os.path.join(output_base_folder, f"{dataset}-VCTK")
    
    # Create output dataset folder if it doesn't exist
    os.makedirs(output_dataset_folder, exist_ok=True)
    
    # Process each voice cloning method
    for method in ["CosyVoice2", "XTTSv2", "YourTTS"]:
        print(f"\nProcessing {method}...")
        
        # Define input and output method folders
        input_method_folder = os.path.join(input_dataset_folder, method)
        output_method_folder = os.path.join(output_dataset_folder, method)
        
        if dataset == "Stroke" and method == "CosyVoice2":
            input_method_folder = os.path.join(input_method_folder, "post_processing_trimmed")

        # Create output method folder if it doesn't exist
        os.makedirs(output_method_folder, exist_ok=True)
        
        # Check if input method folder exists
        if not os.path.exists(input_method_folder):
            print(f"  ⚠️ Input folder tidak ditemukan: {input_method_folder}")
            continue
            
        # Process each audio file
        audio_files = [f for f in os.listdir(input_method_folder) if f.endswith('.wav')]
        
        if not audio_files:
            print(f"  ⚠️ Tidak ada file WAV ditemukan di: {input_method_folder}")
            continue
            
        for audio_file in audio_files:
            # Get the prefix for this method
            prefix = prefix_mapping[method]
            
            # Create new filename with prefix
            if dataset == "ALS":
                # For ALS: A002_cloned.wav -> CV_A002_BBP_cloned.wav (except A015, use PATAKA)
                speaker_id = audio_file.split("_")[0]
                new_filename = f"{prefix}{speaker_id}_VCTK_cloned.wav"
            else:
                # For Stroke: S001_cloned.wav -> CV_S001_cloned.wav
                speaker_id = audio_file.split("_")[0]
                new_filename = f"{prefix}{speaker_id}_VCTK_cloned.wav"

            # Define source and destination paths
            src_path = os.path.join(input_method_folder, audio_file)
            dst_path = os.path.join(output_method_folder, new_filename)
            
            # Copy file with new name
            try:
                shutil.copy2(src_path, dst_path)
                print(f"  ✓ {audio_file} -> {new_filename}")
            except Exception as e:
                print(f"  ✗ Error copying {audio_file}: {e}")


=== Processing ALS dataset ===

Processing CosyVoice2...
  ✓ A016_cloned.wav -> CV_A016_VCTK_cloned.wav
  ✓ A011_cloned.wav -> CV_A011_VCTK_cloned.wav
  ✓ A014_cloned.wav -> CV_A014_VCTK_cloned.wav
  ✓ A002_cloned.wav -> CV_A002_VCTK_cloned.wav
  ✓ A015_cloned.wav -> CV_A015_VCTK_cloned.wav
  ✓ A008_cloned.wav -> CV_A008_VCTK_cloned.wav
  ✓ A010_cloned.wav -> CV_A010_VCTK_cloned.wav
  ✓ A006_cloned.wav -> CV_A006_VCTK_cloned.wav
  ✓ A017_cloned.wav -> CV_A017_VCTK_cloned.wav

Processing XTTSv2...
  ✓ A010_cloned.wav -> XT_A010_VCTK_cloned.wav
  ✓ A006_cloned.wav -> XT_A006_VCTK_cloned.wav
  ✓ A017_cloned.wav -> XT_A017_VCTK_cloned.wav
  ✓ A015_cloned.wav -> XT_A015_VCTK_cloned.wav
  ✓ A008_cloned.wav -> XT_A008_VCTK_cloned.wav
  ✓ A014_cloned.wav -> XT_A014_VCTK_cloned.wav
  ✓ A002_cloned.wav -> XT_A002_VCTK_cloned.wav
  ✓ A016_cloned.wav -> XT_A016_VCTK_cloned.wav
  ✓ A011_cloned.wav -> XT_A011_VCTK_cloned.wav

Processing YourTTS...
  ✓ A008_cloned.wav -> YT_A008_VCTK_cloned.wav
  ✓ 

In [7]:
import os
import subprocess
import cv2
from moviepy.editor import VideoFileClip, AudioFileClip

def combine_video_audio_and_trim(video_path, audio_path, output_path, start_time, duration, target_fps=25):
    """
    Combine video and audio, then trim to specified duration and fps with proper sync
    """
    ffmpeg_executable = "/home/is/lathifgalih-k/musetalk/ffmpeg-4.4-amd64-static/ffmpeg"
    
    # Method 1: Apply start time to both video and audio simultaneously
    cmd = [
        ffmpeg_executable,
        '-ss', str(start_time),    # Apply start time to both inputs
        '-i', video_path,          # Input video
        '-ss', str(start_time),    # Apply start time to audio as well
        '-i', audio_path,          # Input audio
        '-t', str(duration),       # Duration
        '-r', str(target_fps),     # Output FPS
        '-c:v', 'libx264',         # Video codec
        '-c:a', 'aac',             # Audio codec
        '-map', '0:v:0',           # Map video from first input
        '-map', '1:a:0',           # Map audio from second input
        '-shortest',               # Stop when shortest stream ends
        '-avoid_negative_ts', 'make_zero',  # Handle timing issues
        '-y',                      # Overwrite if exists
        output_path
    ]
    
    print(f"Combining and trimming: {os.path.basename(video_path)} + {os.path.basename(audio_path)}")
    print(" ".join(cmd))
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    if result.returncode == 0:
        print(f"✓ Berhasil! Output disimpan di: {output_path}")
        return True
    else:
        print(f"✗ Error: {result.stderr}")
        print("Trying alternative method...")
        
        # Method 2: Alternative approach - trim first, then combine
        return combine_video_audio_alternative(video_path, audio_path, output_path, start_time, duration, target_fps)

def combine_video_audio_alternative(video_path, audio_path, output_path, start_time, duration, target_fps=25):
    """
    Alternative method: trim video and audio separately, then combine
    """
    ffmpeg_executable = "/home/is/lathifgalih-k/musetalk/ffmpeg-4.4-amd64-static/ffmpeg"
    
    # Create temporary files
    temp_video = output_path.replace('.mp4', '_temp_video.mp4')
    temp_audio = output_path.replace('.mp4', '_temp_audio.wav')
    
    try:
        # Step 1: Trim video
        cmd_video = [
            ffmpeg_executable,
            '-i', video_path,
            '-ss', str(start_time),
            '-t', str(duration),
            '-r', str(target_fps),
            '-c:v', 'libx264',
            '-an',  # No audio
            '-y',
            temp_video
        ]
        
        print("Step 1: Trimming video...")
        result_video = subprocess.run(cmd_video, capture_output=True, text=True)
        
        if result_video.returncode != 0:
            print(f"Error trimming video: {result_video.stderr}")
            return False
        
        # Step 2: Trim audio
        cmd_audio = [
            ffmpeg_executable,
            '-i', audio_path,
            '-ss', str(start_time),
            '-t', str(duration),
            '-c:a', 'pcm_s16le',
            '-y',
            temp_audio
        ]
        
        print("Step 2: Trimming audio...")
        result_audio = subprocess.run(cmd_audio, capture_output=True, text=True)
        
        if result_audio.returncode != 0:
            print(f"Error trimming audio: {result_audio.stderr}")
            return False
        
        # Step 3: Combine trimmed video and audio
        cmd_combine = [
            ffmpeg_executable,
            '-i', temp_video,
            '-i', temp_audio,
            '-c:v', 'copy',
            '-c:a', 'aac',
            '-shortest',
            '-y',
            output_path
        ]
        
        print("Step 3: Combining trimmed video and audio...")
        result_combine = subprocess.run(cmd_combine, capture_output=True, text=True)
        
        if result_combine.returncode == 0:
            print(f"✓ Alternative method berhasil! Output disimpan di: {output_path}")
            return True
        else:
            print(f"Error combining: {result_combine.stderr}")
            return False
            
    finally:
        # Clean up temporary files
        if os.path.exists(temp_video):
            os.remove(temp_video)
        if os.path.exists(temp_audio):
            os.remove(temp_audio)

# Process ALS Dataset
print("=== Processing ALS Dataset ===")

# Define paths for ALS
video_dir_als = "/home/is/lathifgalih-k/research_naist/multimodal-clone/dataset-toronto/ALS/Videos"
audio_dir_als = "/home/is/lathifgalih-k/research_naist/multimodal-clone/dataset-toronto/ALS/Audios"
output_dir_als = "/home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video/muse-exp/MuseTalk/data/video/als-combined"

# Create output directory
os.makedirs(output_dir_als, exist_ok=True)

# Define ALS speaker IDs yang ada di cloned audio
input_audio_cloned = "/home/is/lathifgalih-k/research_naist/multimodal-clone/clone-voice-coqui/output_audio/RAW/ALS/XTTSv2"
als_speaker_ids = []
for file in os.listdir(input_audio_cloned):
    if file.endswith("wav"):
        als_speaker_ids.append(file.split("_")[0])

print(f"ALS Speaker IDs: {als_speaker_ids}")

# Process ALS videos
target_duration = 6
target_fps = 25

for speaker_id in als_speaker_ids:
    print(f"\nProcessing ALS Speaker: {speaker_id}")
    
    # Set start time berdasarkan speaker
    if speaker_id == "A008":
        start_time = 12
    else:
        start_time = 9
    
    # Find appropriate video and audio files
    if speaker_id == "A015":
        # For A015, use PATAKA video and audio
        video_pattern = f"{speaker_id}_02_DDK_PATAKA_color"
        search_terms = ["PATAKA", "Pataka", "pataka"]
    else:
        # For others, use BBP_NORMAL
        video_pattern = f"{speaker_id}_02_BBP_NORMAL_color"
        search_terms = ["BBP_NORMAL", "BBP_Normal", "BBP_normal"]
    
    # Find video file (prefer .mp4, fallback to .avi)
    video_file = None
    for ext in ['.mp4', '.avi']:
        video_candidate = f"{video_pattern}{ext}"
        video_path = os.path.join(video_dir_als, video_candidate)
        if os.path.exists(video_path):
            video_file = video_candidate
            break
    
    # Find audio file
    audio_file = None
    for audio_candidate in os.listdir(audio_dir_als):
        if audio_candidate.startswith(speaker_id) and audio_candidate.endswith('.wav'):
            for term in search_terms:
                if term in audio_candidate:
                    audio_file = audio_candidate
                    break
            if audio_file:
                break
    
    print(f"  Found video: {video_file}")
    print(f"  Found audio: {audio_file}")
    
    if video_file and audio_file:
        video_path = os.path.join(video_dir_als, video_file)
        audio_path = os.path.join(audio_dir_als, audio_file)
        
        # Create output filename
        if speaker_id == "A015":
            output_filename = f"{speaker_id}_PATAKA_combined_25fps_6sec.mp4"
        else:
            output_filename = f"{speaker_id}_BBP_combined_25fps_6sec.mp4"
        
        output_path = os.path.join(output_dir_als, output_filename)
        
        # Combine and trim
        success = combine_video_audio_and_trim(
            video_path, audio_path, output_path, 
            start_time, target_duration, target_fps
        )
        
        if success:
            # Verify output
            cap = cv2.VideoCapture(output_path)
            fps = cap.get(cv2.CAP_PROP_FPS)
            frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            duration = frame_count / fps if fps > 0 else 0
            cap.release()
            
            print(f"  - Output FPS: {fps}")
            print(f"  - Duration: {duration:.2f} seconds")
            print(f"  - Frame count: {frame_count}")
        
    else:
        print(f"  ✗ Files not found for {speaker_id}")

print("\n=== Processing Stroke Dataset ===")

# Define paths for Stroke
video_dir_stroke = "/home/is/lathifgalih-k/research_naist/multimodal-clone/dataset-toronto/Stroke/Videos"
audio_dir_stroke = "/home/is/lathifgalih-k/research_naist/multimodal-clone/dataset-toronto/Stroke/Audios"
output_dir_stroke = "/home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video/muse-exp/MuseTalk/data/video/stroke-combined"

# Create output directory
os.makedirs(output_dir_stroke, exist_ok=True)

# Define Stroke speaker IDs
input_audio_cloned_stroke = "/home/is/lathifgalih-k/research_naist/multimodal-clone/clone-voice-coqui/output_audio/RAW/Stroke/XTTSv2"
stroke_speaker_ids = []
for file in os.listdir(input_audio_cloned_stroke):
    if file.endswith("wav"):
        stroke_speaker_ids.append(file.split("_")[0])

print(f"Stroke Speaker IDs: {stroke_speaker_ids}")

# Process Stroke videos
for speaker_id in stroke_speaker_ids:
    print(f"\nProcessing Stroke Speaker: {speaker_id}")
    
    # Set start time berdasarkan speaker
    if speaker_id == "S013":
        start_time = 9
    elif speaker_id == "S002":
        start_time = 4.5
    else:
        start_time = 4
    
    # Find appropriate video and audio files
    if speaker_id in ["S001", "S002"]:
        # For S001 and S002, use PA video and audio
        video_pattern = f"{speaker_id}_02_DDK_PA_color"
        search_term = "PA"
    else:
        # For others, use BBP_NORMAL
        video_pattern = f"{speaker_id}_02_BBP_NORMAL_color"
        search_term = "BBP_NORMAL"
    
    # Find video file (.avi extension for Stroke dataset)
    video_file = f"{video_pattern}.avi"
    video_path = os.path.join(video_dir_stroke, video_file)
    
    # Find audio file
    audio_file = None
    for audio_candidate in os.listdir(audio_dir_stroke):
        if (audio_candidate.startswith(speaker_id) and 
            search_term in audio_candidate and 
            (audio_candidate.endswith('.wav') or audio_candidate.endswith('.WAV'))):
            audio_file = audio_candidate
            break
    
    print(f"  Found video: {os.path.exists(video_path)}")
    print(f"  Found audio: {audio_file}")
    
    if os.path.exists(video_path) and audio_file:
        audio_path = os.path.join(audio_dir_stroke, audio_file)
        
        # Create output filename
        if speaker_id in ["S001", "S002"]:
            output_filename = f"{speaker_id}_PA_combined_25fps_6sec.mp4"
        else:
            output_filename = f"{speaker_id}_BBP_combined_25fps_6sec.mp4"
        
        output_path = os.path.join(output_dir_stroke, output_filename)
        
        # Combine and trim
        success = combine_video_audio_and_trim(
            video_path, audio_path, output_path, 
            start_time, target_duration, target_fps
        )
        
        if success:
            # Verify output
            cap = cv2.VideoCapture(output_path)
            fps = cap.get(cv2.CAP_PROP_FPS)
            frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            duration = frame_count / fps if fps > 0 else 0
            cap.release()
            
            print(f"  - Output FPS: {fps}")
            print(f"  - Duration: {duration:.2f} seconds")
            print(f"  - Frame count: {frame_count}")
        
    else:
        print(f"  ✗ Files not found for {speaker_id}")

print("\n=== Processing completed ===")
print(f"ALS output directory: {output_dir_als}")
print(f"Stroke output directory: {output_dir_stroke}")

=== Processing ALS Dataset ===
ALS Speaker IDs: ['A017', 'A006', 'A010', 'A008', 'A015', 'A002', 'A014', 'A011', 'A016']

Processing ALS Speaker: A017
  Found video: A017_02_BBP_NORMAL_color.mp4
  Found audio: A017_02_BBP_Normal.wav
Combining and trimming: A017_02_BBP_NORMAL_color.mp4 + A017_02_BBP_Normal.wav
/home/is/lathifgalih-k/musetalk/ffmpeg-4.4-amd64-static/ffmpeg -ss 9 -i /home/is/lathifgalih-k/research_naist/multimodal-clone/dataset-toronto/ALS/Videos/A017_02_BBP_NORMAL_color.mp4 -ss 9 -i /home/is/lathifgalih-k/research_naist/multimodal-clone/dataset-toronto/ALS/Audios/A017_02_BBP_Normal.wav -t 6 -r 25 -c:v libx264 -c:a aac -map 0:v:0 -map 1:a:0 -shortest -avoid_negative_ts make_zero -y /home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video/muse-exp/MuseTalk/data/video/als-combined/A017_BBP_combined_25fps_6sec.mp4
✓ Berhasil! Output disimpan di: /home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video/muse-exp/MuseTalk/data/video/als-combined/A017_BBP_com

In [8]:
import os
import subprocess
import cv2
from moviepy.editor import VideoFileClip, AudioFileClip

def combine_video_audio_and_trim(video_path, audio_path, output_path, start_time, duration, target_fps=25):
    """
    Combine video and audio, then trim to specified duration and fps with proper sync
    """
    ffmpeg_executable = "/home/is/lathifgalih-k/musetalk/ffmpeg-4.4-amd64-static/ffmpeg"
    
    # Method 1: Apply start time to both video and audio simultaneously
    cmd = [
        ffmpeg_executable,
        '-ss', str(start_time),    # Apply start time to both inputs
        '-i', video_path,          # Input video
        '-ss', str(start_time),    # Apply start time to audio as well
        '-i', audio_path,          # Input audio
        '-t', str(duration),       # Duration
        '-r', str(target_fps),     # Output FPS
        '-c:v', 'libx264',         # Video codec
        '-c:a', 'aac',             # Audio codec
        '-map', '0:v:0',           # Map video from first input
        '-map', '1:a:0',           # Map audio from second input
        '-shortest',               # Stop when shortest stream ends
        '-avoid_negative_ts', 'make_zero',  # Handle timing issues
        '-y',                      # Overwrite if exists
        output_path
    ]
    
    print(f"Combining and trimming: {os.path.basename(video_path)} + {os.path.basename(audio_path)}")
    print(" ".join(cmd))
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    if result.returncode == 0:
        print(f"✓ Berhasil! Output disimpan di: {output_path}")
        return True
    else:
        print(f"✗ Error: {result.stderr}")
        print("Trying alternative method...")
        
        # Method 2: Alternative approach - trim first, then combine
        return combine_video_audio_alternative(video_path, audio_path, output_path, start_time, duration, target_fps)

def combine_video_audio_alternative(video_path, audio_path, output_path, start_time, duration, target_fps=25):
    """
    Alternative method: trim video and audio separately, then combine
    """
    ffmpeg_executable = "/home/is/lathifgalih-k/musetalk/ffmpeg-4.4-amd64-static/ffmpeg"
    
    # Create temporary files
    temp_video = output_path.replace('.mp4', '_temp_video.mp4')
    temp_audio = output_path.replace('.mp4', '_temp_audio.wav')
    
    try:
        # Step 1: Trim video
        cmd_video = [
            ffmpeg_executable,
            '-i', video_path,
            '-ss', str(start_time),
            '-t', str(duration),
            '-r', str(target_fps),
            '-c:v', 'libx264',
            '-an',  # No audio
            '-y',
            temp_video
        ]
        
        print("Step 1: Trimming video...")
        result_video = subprocess.run(cmd_video, capture_output=True, text=True)
        
        if result_video.returncode != 0:
            print(f"Error trimming video: {result_video.stderr}")
            return False
        
        # Step 2: Trim audio
        cmd_audio = [
            ffmpeg_executable,
            '-i', audio_path,
            '-ss', str(start_time),
            '-t', str(duration),
            '-c:a', 'pcm_s16le',
            '-y',
            temp_audio
        ]
        
        print("Step 2: Trimming audio...")
        result_audio = subprocess.run(cmd_audio, capture_output=True, text=True)
        
        if result_audio.returncode != 0:
            print(f"Error trimming audio: {result_audio.stderr}")
            return False
        
        # Step 3: Combine trimmed video and audio
        cmd_combine = [
            ffmpeg_executable,
            '-i', temp_video,
            '-i', temp_audio,
            '-c:v', 'copy',
            '-c:a', 'aac',
            '-shortest',
            '-y',
            output_path
        ]
        
        print("Step 3: Combining trimmed video and audio...")
        result_combine = subprocess.run(cmd_combine, capture_output=True, text=True)
        
        if result_combine.returncode == 0:
            print(f"✓ Alternative method berhasil! Output disimpan di: {output_path}")
            return True
        else:
            print(f"Error combining: {result_combine.stderr}")
            return False
            
    finally:
        # Clean up temporary files
        if os.path.exists(temp_video):
            os.remove(temp_video)
        if os.path.exists(temp_audio):
            os.remove(temp_audio)

# Process ALS Dataset
print("=== Processing ALS Dataset ===")

# Define paths for ALS
video_dir_als = "/home/is/lathifgalih-k/research_naist/multimodal-clone/dataset-toronto/ALS/Videos"
audio_dir_als = "/home/is/lathifgalih-k/research_naist/multimodal-clone/dataset-toronto/ALS/Audios"
output_dir_als = "/home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video/muse-exp/MuseTalk/data/video/als-combined"

# Create output directory
os.makedirs(output_dir_als, exist_ok=True)

# Define ALS speaker IDs yang ada di cloned audio
input_audio_cloned = "/home/is/lathifgalih-k/research_naist/multimodal-clone/clone-voice-coqui/output_audio/RAW/ALS/XTTSv2"
als_speaker_ids = []
for file in os.listdir(input_audio_cloned):
    if file.endswith("wav"):
        als_speaker_ids.append(file.split("_")[0])

print(f"ALS Speaker IDs: {als_speaker_ids}")

# Process ALS videos
target_duration = 6
target_fps = 25

for speaker_id in als_speaker_ids:
    print(f"\nProcessing ALS Speaker: {speaker_id}")
    
    # Set start time berdasarkan speaker
    if speaker_id == "A008":
        start_time = 12
    else:
        start_time = 9
    
    # Find appropriate video and audio files
    if speaker_id == "A015":
        # For A015, use PATAKA video and audio
        video_pattern = f"{speaker_id}_02_DDK_PATAKA_color"
        search_terms = ["PATAKA", "Pataka", "pataka"]
    else:
        # For others, use BBP_NORMAL
        video_pattern = f"{speaker_id}_02_BBP_NORMAL_color"
        search_terms = ["BBP_NORMAL", "BBP_Normal", "BBP_normal"]
    
    # Find video file (prefer .mp4, fallback to .avi)
    video_file = None
    for ext in ['.mp4', '.avi']:
        video_candidate = f"{video_pattern}{ext}"
        video_path = os.path.join(video_dir_als, video_candidate)
        if os.path.exists(video_path):
            video_file = video_candidate
            break
    
    # Find audio file
    audio_file = None
    for audio_candidate in os.listdir(audio_dir_als):
        if audio_candidate.startswith(speaker_id) and audio_candidate.endswith('.wav'):
            for term in search_terms:
                if term in audio_candidate:
                    audio_file = audio_candidate
                    break
            if audio_file:
                break
    
    print(f"  Found video: {video_file}")
    print(f"  Found audio: {audio_file}")
    
    if video_file and audio_file:
        video_path = os.path.join(video_dir_als, video_file)
        audio_path = os.path.join(audio_dir_als, audio_file)
        
        # Create output filename
        if speaker_id == "A015":
            output_filename = f"{speaker_id}_PATAKA_combined_25fps_6sec.mp4"
        else:
            output_filename = f"{speaker_id}_BBP_combined_25fps_6sec.mp4"
        
        output_path = os.path.join(output_dir_als, output_filename)
        
        # Combine and trim
        success = combine_video_audio_and_trim(
            video_path, audio_path, output_path, 
            start_time, target_duration, target_fps
        )
        
        if success:
            # Verify output
            cap = cv2.VideoCapture(output_path)
            fps = cap.get(cv2.CAP_PROP_FPS)
            frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            duration = frame_count / fps if fps > 0 else 0
            cap.release()
            
            print(f"  - Output FPS: {fps}")
            print(f"  - Duration: {duration:.2f} seconds")
            print(f"  - Frame count: {frame_count}")
        
    else:
        print(f"  ✗ Files not found for {speaker_id}")

print("\n=== Processing Stroke Dataset ===")

# Define paths for Stroke
video_dir_stroke = "/home/is/lathifgalih-k/research_naist/multimodal-clone/dataset-toronto/Stroke/Videos"
audio_dir_stroke = "/home/is/lathifgalih-k/research_naist/multimodal-clone/dataset-toronto/Stroke/Audios"
output_dir_stroke = "/home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video/muse-exp/MuseTalk/data/video/stroke-combined"

# Create output directory
os.makedirs(output_dir_stroke, exist_ok=True)

# Define Stroke speaker IDs
input_audio_cloned_stroke = "/home/is/lathifgalih-k/research_naist/multimodal-clone/clone-voice-coqui/output_audio/RAW/Stroke/XTTSv2"
stroke_speaker_ids = []
for file in os.listdir(input_audio_cloned_stroke):
    if file.endswith("wav"):
        stroke_speaker_ids.append(file.split("_")[0])

print(f"Stroke Speaker IDs: {stroke_speaker_ids}")

# Process Stroke videos
for speaker_id in stroke_speaker_ids:
    print(f"\nProcessing Stroke Speaker: {speaker_id}")
    
    # Set start time berdasarkan speaker
    if speaker_id == "S013":
        start_time = 9
    elif speaker_id == "S002":
        start_time = 4.5
    else:
        start_time = 4
    
    # Find appropriate video and audio files
    if speaker_id in ["S001", "S002"]:
        # For S001 and S002, use PA video and audio
        video_pattern = f"{speaker_id}_02_DDK_PA_color"
        search_term = "PA"
    else:
        # For others, use BBP_NORMAL
        video_pattern = f"{speaker_id}_02_BBP_NORMAL_color"
        search_term = "BBP_NORMAL"
    
    # Find video file (.avi extension for Stroke dataset)
    video_file = f"{video_pattern}.avi"
    video_path = os.path.join(video_dir_stroke, video_file)
    
    # Find audio file
    audio_file = None
    for audio_candidate in os.listdir(audio_dir_stroke):
        if (audio_candidate.startswith(speaker_id) and 
            search_term in audio_candidate and 
            (audio_candidate.endswith('.wav') or audio_candidate.endswith('.WAV'))):
            audio_file = audio_candidate
            break
    
    print(f"  Found video: {os.path.exists(video_path)}")
    print(f"  Found audio: {audio_file}")
    
    if os.path.exists(video_path) and audio_file:
        audio_path = os.path.join(audio_dir_stroke, audio_file)
        
        # Create output filename
        if speaker_id in ["S001", "S002"]:
            output_filename = f"{speaker_id}_PA_combined_25fps_6sec.mp4"
        else:
            output_filename = f"{speaker_id}_BBP_combined_25fps_6sec.mp4"
        
        output_path = os.path.join(output_dir_stroke, output_filename)
        
        # Combine and trim
        success = combine_video_audio_and_trim(
            video_path, audio_path, output_path, 
            start_time, target_duration, target_fps
        )
        
        if success:
            # Verify output
            cap = cv2.VideoCapture(output_path)
            fps = cap.get(cv2.CAP_PROP_FPS)
            frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            duration = frame_count / fps if fps > 0 else 0
            cap.release()
            
            print(f"  - Output FPS: {fps}")
            print(f"  - Duration: {duration:.2f} seconds")
            print(f"  - Frame count: {frame_count}")
        
    else:
        print(f"  ✗ Files not found for {speaker_id}")

print("\n=== Processing completed ===")
print(f"ALS output directory: {output_dir_als}")
print(f"Stroke output directory: {output_dir_stroke}")

=== Processing ALS Dataset ===
ALS Speaker IDs: ['A017', 'A006', 'A010', 'A008', 'A015', 'A002', 'A014', 'A011', 'A016']

Processing ALS Speaker: A017
  Found video: A017_02_BBP_NORMAL_color.mp4
  Found audio: A017_02_BBP_Normal.wav
Combining and trimming: A017_02_BBP_NORMAL_color.mp4 + A017_02_BBP_Normal.wav
/home/is/lathifgalih-k/musetalk/ffmpeg-4.4-amd64-static/ffmpeg -ss 9 -i /home/is/lathifgalih-k/research_naist/multimodal-clone/dataset-toronto/ALS/Videos/A017_02_BBP_NORMAL_color.mp4 -ss 9 -i /home/is/lathifgalih-k/research_naist/multimodal-clone/dataset-toronto/ALS/Audios/A017_02_BBP_Normal.wav -t 6 -r 25 -c:v libx264 -c:a aac -map 0:v:0 -map 1:a:0 -shortest -avoid_negative_ts make_zero -y /home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video/muse-exp/MuseTalk/data/video/als-combined/A017_BBP_combined_25fps_6sec.mp4
✓ Berhasil! Output disimpan di: /home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video/muse-exp/MuseTalk/data/video/als-combined/A017_BBP_com